In [0]:
# analytics/make_charts.py
import os
import pandas as pd
import matplotlib.pyplot as plt

# Expect a Spark session to exist (Databricks job / notebook). If running locally, create SparkSession first.
from pyspark.sql import SparkSession
spark = SparkSession.getActiveSession() or SparkSession.builder.getOrCreate()

# Config
SCHEMA = "demo_finance"
OUT_DIR_LOCAL = "./assets"
OUT_DIR_DBFS  = "/dbfs/FileStore/finance_demo"

os.makedirs(OUT_DIR_LOCAL, exist_ok=True)
os.makedirs(OUT_DIR_DBFS, exist_ok=True)

def save(fig, name):
    p1 = f"{OUT_DIR_LOCAL}/{name}.png"
    p2 = f"{OUT_DIR_DBFS}/{name}.png"
    fig.savefig(p1, dpi=160, bbox_inches="tight")
    fig.savefig(p2, dpi=160, bbox_inches="tight")
    print("Saved:", p1, "and", p2)

def load():
    spark.sql(f"USE {SCHEMA}")
    df_pay = spark.table("gold_payments_monthly").toPandas().copy()
    df_cb  = spark.table("gold_chargeback_rate").toPandas().copy()
    for df in (df_pay, df_cb):
        df["month_date"] = pd.to_datetime(df["month"] + "-01")
    return df_pay.sort_values("month_date"), df_cb.sort_values("month_date")

def chart_payments(df):
    fig = plt.figure(figsize=(10,4.5))
    plt.plot(df["month_date"], df["payments"], label="Payments")
    plt.plot(df["month_date"], df["settled"],  label="Settled")
    plt.plot(df["month_date"], df["failed"],   label="Failed")
    plt.title("Monthly Payments (all / settled / failed)")
    plt.xlabel("Month"); plt.ylabel("Count")
    plt.xticks(rotation=45, ha="right"); plt.legend(); plt.tight_layout()
    save(fig, "payments_trend")

def chart_chargebacks(df):
    fig = plt.figure(figsize=(10,4.5))
    plt.plot(df["month_date"], df["chargeback_rate"])
    plt.title("Monthly Chargeback Rate")
    plt.xlabel("Month"); plt.ylabel("Rate")
    plt.xticks(rotation=45, ha="right"); plt.tight_layout()
    save(fig, "chargeback_rate")

def chart_revenue(df):
    fig = plt.figure(figsize=(10,4.5))
    plt.bar(df["month_date"], df["gross_revenue"])
    plt.title("Monthly Gross Revenue (settled only)")
    plt.xlabel("Month"); plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right"); plt.tight_layout()
    save(fig, "gross_revenue")

if __name__ == "__main__":
    df_pay, df_cb = load()
    chart_payments(df_pay)
    chart_chargebacks(df_cb)
    chart_revenue(df_pay)
    print("✅ Charts generated.")
